In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Fake News Classifier Using LSTM


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df = df.dropna()

In [6]:
## Get the Independent Features

X = df.drop('label',axis=1)

In [7]:
y = df['label']

In [8]:
X.shape

(18285, 4)

In [9]:
import tensorflow as tf

In [11]:
tf.__version__

'2.15.0'

In [12]:
from tensorflow.keras.layers import  Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
### Vocabulary size
voc_size=5000

Onehot Representation

In [14]:
messages = X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [16]:
messages.reset_index(inplace=True)

In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[497, 1362, 1792, 523, 3476, 3394, 1778, 4381, 1064, 3920],
 [2932, 4211, 2736, 4782, 2961, 3994, 3970],
 [1452, 2999, 3413, 2461],
 [4538, 2979, 2986, 2405, 2811, 3522],
 [3246, 2961, 2634, 3011, 2198, 2653, 2961, 3196, 3011, 3970],
 [2602,
  819,
  4268,
  1388,
  857,
  4900,
  3794,
  2041,
  2656,
  3747,
  3019,
  4868,
  3287,
  3865,
  3970],
 [699, 1661, 4873, 697, 2937, 2262, 1376, 3767, 112, 795, 4567],
 [4688, 78, 592, 2686, 3628, 595, 4900, 2369, 112, 795, 4567],
 [2886, 3936, 2616, 1939, 1802, 4899, 3508, 170, 4900, 2391],
 [2714, 2969, 3464, 2594, 502, 2557, 122, 294],
 [337, 4101, 3014, 2904, 699, 1886, 2231, 3851, 4206, 3349, 4257],
 [2405, 119, 3476, 4899, 4900, 3628],
 [2328, 1155, 3622, 1860, 3765, 4256, 15, 3750, 2363],
 [3643, 3329, 4810, 4328, 2376, 2382, 3355, 112, 795, 4567],
 [858, 3122, 4945, 2047, 949, 112, 795, 4567],
 [719, 2657, 676, 1442, 1739, 168, 3992, 1820, 4772, 1790],
 [3576, 2849, 4211],
 [3726, 4237, 4282, 18, 4900, 4011, 1775, 3970],
 [4471, 47

Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4381 1064 3920]
 [   0    0    0 ... 2961 3994 3970]
 [   0    0    0 ... 2999 3413 2461]
 ...
 [   0    0    0 ...  112  795 4567]
 [   0    0    0 ... 4019    2 2040]
 [   0    0    0 ...  178 1655 1231]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  497,
       1362, 1792,  523, 3476, 3394, 1778, 4381, 1064, 3920], dtype=int32)

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


Model Training

In [29]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 16s 66ms/step - loss: 0.3309 - accuracy: 0.8505 - val_loss: 0.1929 - val_accuracy: 0.9203
Epoch 2/10
192/192 [==============================] - 5s 26ms/step - loss: 0.1374 - accuracy: 0.9463 - val_loss: 0.1892 - val_accuracy: 0.9201
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0961 - accuracy: 0.9638 - val_loss: 0.2265 - val_accuracy: 0.9203
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0685 - accuracy: 0.9764 - val_loss: 0.2335 - val_accuracy: 0.9195
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0438 - accuracy: 0.9859 - val_loss: 0.2573 - val_accuracy: 0.9176
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0270 - accuracy: 0.9912 - val_loss: 0.3445 - val_accuracy: 0.9147
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0160 - accuracy: 0.9952 - val_loss: 0.3717 - val_accuracy: 0.9138
E

Adding Dropout

In [30]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Performance Metrics And Accuracy


In [37]:
y_pred = model.predict(X_test)


189/189 [==============================] - 0s 2ms/step


In [38]:
from sklearn.metrics import confusion_matrix

In [39]:
confusion_matrix(y_test,y_pred)

ValueError: ignored

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

ValueError: ignored